<a href="https://colab.research.google.com/github/Bech007/Text-Skew/blob/master/GenerateurPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
from pathlib import Path

 

drive.mount('/content/drive')
root_path = Path('/content/drive/My Drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!pip install pdf2image

In [0]:
import cv2
import pandas as pd
import numpy as np
from fpdf import FPDF
import random
from datetime import date
from datetime import timedelta
import time
import datetime
import string

In [0]:
def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.digits
    return ''.join(random.choice(letters) for i in range(stringLength))

In [0]:
dir ='/content/drive/My Drive/PFE Bechir (Rec Doc)/Data/DatasetPDF/'
def gen(itr):
    
    data  = pd.read_csv("/content/drive/My Drive/PFE Bechir (Rec Doc)/Data/carte-des-pharmacies-de-paris.csv", sep=';')
    data1 = pd.read_csv("/content/drive/My Drive/PFE Bechir (Rec Doc)/Data/medica.csv", sep=',')
    data2 = pd.read_csv("/content/drive/My Drive/PFE Bechir (Rec Doc)/Data/data-medecins.csv", sep=';')
    data["numvoie"] = pd.to_numeric(data["numvoie"])
    data['telephone']= pd.to_numeric(data['telephone'])
    data['adress']= data["numvoie"].astype(str)+" "+ data['typvoie'].astype(str) +" DU "+data['voie']
    data['commune']= data["cp"].astype(str)+" "+ data['commune']
    data2['NomMedecin']= data2["Nom du médecin"]+" "+ data2['Prénom du médecin']
    nom_pharmacie=list(data['rs'])
    commune=list(data['commune'])
    NomMedecin = list(data2['NomMedecin'])
    tel=list(data['telephone'].map(str))
    adresse=list(data['adress'])
    CodeMedic = list(data1['CODE'])
    res1 = [str(i)for i in CodeMedic]
    res1
    NomMedic=list(data1['NOM'])
    PrixMedic=list(data1['PRIX_BR'])
    TR=list(data1['TAUX_REMBOURSEMENT'])
    for i in range(len(adresse)) :
        adresse[i] = adresse[i].replace('.0', '')

    for i in range(len(tel)) :
        tel[i] = tel[i].replace('.0', '')


    n_rows = 1
    start_time = "01/01/2000"
    end_time = "31/12/2019"

    date2int = lambda s: time.mktime(datetime.datetime.strptime(s,"%d/%m/%Y").timetuple())
    int2date = lambda s: datetime.datetime.fromtimestamp(s).strftime('%d/%m/%Y')

    start_time = date2int(start_time)
    end_time = date2int(end_time)
    random_ints = np.random.randint(low=start_time, high=end_time, size=(n_rows,1))
    random_dates = np.apply_along_axis(int2date, 1, random_ints).reshape(n_rows,1)

    res=str(random_dates)
    res=res.replace("[['","")
    res = res.replace("']]","")
    #####
    res

    qte= ['1','2','3','4']
    A=['X','T','E']
    monpdf= FPDF(orientation='P', unit='mm', format='A4')
    monpdf.add_page()
    monpdf.set_font("Courier", size=5)
    monpdf.cell(200,2,txt=random.choice(nom_pharmacie)+"              "+"NumFact  :"+"   " +randomString(stringLength=6), ln=1, align ="L")
    monpdf.cell(200,2,txt=random.choice(adresse)+"                "+"Date Fact:"+"  "+res, ln=1, align ="L")
    monpdf.cell(200,2,txt=random.choice(commune)+"                             "+"Date Ord :"+"  "+res, ln=1, align ="L")
    monpdf.cell(200,2,txt=random.choice(tel)+"             "+"Medicin :"+" "+random.choice(NomMedecin), ln=1, align ="L")
    monpdf.cell(200,2,txt="TEL :"+" 0"+random.choice(tel)+"    "+"Code OP :"+randomString(stringLength=2), ln=1, align ="L")
    monpdf.cell(200,2,txt="INSEE:"+randomString(stringLength=15)+"       "+"Malade: "+random.choice(A), ln=1, align ="L")
    monpdf.cell(200,2,txt=random.choice(adresse)+"      "+random.choice(commune), ln=1, align ="L")
    monpdf.cell(200,2,txt=random.choice(A), ln=1, align ="L")

    data = [['QTE', 'Designation', 'CIP', 'Prix/U','BaseSS','HD','TxSS'],
            [random.choice(qte), random.choice(NomMedic), random.choice(res1), random.choice(PrixMedic),'-','1,02',random.choice(TR)],
            [random.choice(qte), random.choice(NomMedic), random.choice(res1), random.choice(PrixMedic),'-','1,02',random.choice(TR)],
            [random.choice(qte), random.choice(NomMedic), random.choice(res1), random.choice(PrixMedic),'-','1,02',random.choice(TR)]
            ]
    sizes=[0,0,0,0,0,0,0]
    for row in data:
      for j in range(len(row)):
        if (len(row[j])>sizes[j]):
          sizes[j]=len(row[j])

    spacing=2


    row_height = monpdf.font_size
    for row in data:
        for indice,item in enumerate(row):
            monpdf.cell(sizes[indice] +2, row_height*spacing,
                        txt=item, border=0, align="L")
        monpdf.ln(row_height*spacing)


    monpdf.cell(200,2,txt="Ces montants s'entendent avant application, le cas échéant,", ln=1, align ="L")
    monpdf.cell(200,2,txt="de la franchise prévue à l'article L. 322­2 du code de la ",ln=1 , align="L")
    monpdf.cell(200,2,txt="sécurité sociale et indiquée sur le relevé de prestations.",ln=1 , align="L")
    monpdf.output(dir+'f'+str(itr)+'.pdf')
    print('f'+str(itr)+'.pdf')
    return data


In [13]:
u = 0
for i in range (300,401):
    u = gen(i)

f300.pdf
f301.pdf
f302.pdf
f303.pdf
f304.pdf
f305.pdf
f306.pdf
f307.pdf
f308.pdf
f309.pdf
f310.pdf
f311.pdf
f312.pdf
f313.pdf
f314.pdf
f315.pdf
f316.pdf
f317.pdf
f318.pdf
f319.pdf
f320.pdf
f321.pdf
f322.pdf
f323.pdf
f324.pdf
f325.pdf
f326.pdf
f327.pdf
f328.pdf
f329.pdf
f330.pdf
f331.pdf
f332.pdf
f333.pdf
f334.pdf
f335.pdf
f336.pdf
f337.pdf
f338.pdf
f339.pdf
f340.pdf
f341.pdf
f342.pdf
f343.pdf
f344.pdf
f345.pdf
f346.pdf
f347.pdf
f348.pdf
f349.pdf
f350.pdf
f351.pdf
f352.pdf
f353.pdf
f354.pdf
f355.pdf
f356.pdf
f357.pdf
f358.pdf
f359.pdf
f360.pdf
f361.pdf
f362.pdf
f363.pdf
f364.pdf
f365.pdf
f366.pdf
f367.pdf
f368.pdf
f369.pdf
f370.pdf
f371.pdf
f372.pdf
f373.pdf
f374.pdf
f375.pdf
f376.pdf
f377.pdf
f378.pdf
f379.pdf
f380.pdf
f381.pdf
f382.pdf
f383.pdf
f384.pdf
f385.pdf
f386.pdf
f387.pdf
f388.pdf
f389.pdf
f390.pdf
f391.pdf
f392.pdf
f393.pdf
f394.pdf
f395.pdf
f396.pdf
f397.pdf
f398.pdf
f399.pdf
f400.pdf


In [15]:
import os
from pdf2image import convert_from_path
pdf_dir = r"/content/drive/My Drive/PFE Bechir (Rec Doc)/Data/DatasetPDF/"
out_dir ="/content/drive/My Drive/PFE Bechir (Rec Doc)/Data/DatasetIMG/"
os.chdir(pdf_dir)
for pdf_file in os.listdir(pdf_dir):
  if pdf_file.endswith(".pdf"):
    pages = convert_from_path(pdf_file, 300)
    pdf_file = pdf_file[:-4]
    for page in pages:
      page.save("/content/drive/My Drive/PFE Bechir (Rec Doc)/Data/DatasetIMG/"+".jpg", "JPEG")

PDFInfoNotInstalledError: ignored